In [1]:
import pandas as pd
import numpy as np
import theano
import theano.tensor as T
import lasagne
import os
import re
import time

import matplotlib.pyplot as plt
%matplotlib inline

import re
import pandas as pd
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
from sklearn.feature_extraction.text import _document_frequency
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, StratifiedKFold
import scipy.sparse as sp
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelBinarizer
from keras import backend as K
#from keras.regularizers import l2, activity_l2
from keras.layers.embeddings import Embedding
from keras.layers import Input, Dense, Dropout
from keras.models import Model
from keras.layers.recurrent import LSTM
from keras.optimizers import Nadam
from sklearn.svm import LinearSVC
from sklearn.cluster import KMeans
import word2vecReader
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.casual import TweetTokenizer
from sklearn.ensemble import RandomForestClassifier as RandForest
from sklearn.svm import SVC
from nltk.tokenize import RegexpTokenizer

Using cuDNN version 5110 on context None
Mapped name None to device cuda: GeForce GTX 960 (0000:01:00.0)
/home/svetlana/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using Theano backend.


In [2]:
from __future__ import division

In [3]:
if os.path.exists('business.json'):
    pass
else:
    !wget https://s3.amazonaws.com/cloudfinalprojectsubmission/business.json

if os.path.exists('review.json'):
    pass
else:
    !wget https://s3.amazonaws.com/cloudfinalprojectsubmission/review.json

In [4]:
chunk_size = 100000


usa_business_idx = []
business_reader = pd.read_table('business.json', header=None, chunksize=chunk_size)
categories = pd.DataFrame()

try:
    for i, chunks in enumerate(business_reader):

        start_index = chunk_size * i

        for index in range(start_index, start_index + len(chunks)):

            data_json = pd.read_json(chunks[0][index], typ='series')
            if data_json['city'] in ['Pittsburgh', 'Charlotte', 'Urbana-Champaign',
                                     'Phoenix', 'Las Vegas', 'Madison']:
                usa_business_idx.append(data_json['business_id'])
                categories = categories.append(data_json[['business_id', 'categories']], ignore_index=True)
except:
    print(i)

In [5]:
categories = categories.drop_duplicates('business_id')
categories = categories.set_index('business_id')

In [6]:
categories.shape

(11487, 1)

In [7]:
review_reader = pd.read_table('review.json', header=None, chunksize=chunk_size)
df = pd.DataFrame()


try:
    for i, chunks in enumerate(review_reader):

        start_index = chunk_size * i
        for index in range(start_index, start_index + len(chunks)):
                data_json = pd.read_json(chunks[0][index], typ='series')
                bus_id = data_json['business_id']
                if (bus_id in usa_business_idx) and (u'Restaurants' in categories.loc[bus_id].categories):
                    df = df.append(data_json[['text', 'stars']], ignore_index=True)
                    if data_json['stars'] < 4.:
                        df = df.append(data_json[['text', 'stars']], ignore_index=True)
except:
    pass

In [8]:
label_encoder = LabelEncoder()

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['stars'], test_size=0.33, random_state=42)

In [11]:
import os

os.chdir("/media/svetlana/DATA/dir/this")

In [12]:
from gensim.models.keyedvectors import KeyedVectors
from word2vecReader import Word2Vec
import gensim

In [13]:
tokenizer = TweetTokenizer(reduce_len=True)

In [14]:
fname = 'word2vec_twitter_model/word2vec_twitter_model.bin'
word2vec_model = Word2Vec.load_word2vec_format(fname, binary=True)  # C binary format

In [15]:
from gensim.models.keyedvectors import KeyedVectors
from word2vecReader import Word2Vec
import gensim
#tokenizer = RegexpTokenizer(r'\w+')
tokenizer = TweetTokenizer(reduce_len=True)
fname = 'word2vec_twitter_model/word2vec_twitter_model.bin'
word2vec_model = Word2Vec.load_word2vec_format(fname, binary=True)  # C binary format
max_words_count = max((len(tokenizer.tokenize(sent)) for sent in X_train.values))
word2vec_dim = 400 # ?

In [16]:
from sklearn.preprocessing import LabelBinarizer
group_lb = LabelBinarizer()

In [17]:
from functools import reduce
words_set = set(reduce(lambda x, y: x + y, map(lambda x: tokenizer.tokenize(x), X_train.values.tolist())))

index_dict = {item: (number + 1) for number, item in enumerate(words_set)}
n_symbols = len(index_dict) + 1 # adding 1 to account for 0th index (for masking)
embedding_weights = np.zeros((len(index_dict) + 1, word2vec_dim))
for word, index in index_dict.items():
    if word in word2vec_model:
        embedding_weights[index,:] = word2vec_model[word] / np.linalg.norm(word2vec_model[word])
    else:
        embedding_weights[index, :] = np.zeros(word2vec_dim)

In [18]:
del word2vec_model

In [19]:
max_words_count = 30
def for_embedding(text, word_index):
    words = tokenizer.tokenize(text)
    words_idxes = np.zeros(max_words_count)
    i = 0
    for word in words:
        if (word in word_index) and (i < max_words_count - 1):
            words_idxes[i] = word_index[word]
            i += 1
    return words_idxes

In [20]:
X_train = np.array(X_train.apply(lambda x: for_embedding(x, index_dict)).tolist())

In [21]:
y_train = group_lb.fit_transform(y_train)

In [22]:
X_test = np.array(X_test.apply(lambda x: for_embedding(x, index_dict)).tolist())
y_test = group_lb.fit_transform(y_test)

tokenizer = RegexpTokenizer(r'\w+')

In [23]:
from keras.models import Sequential
from keras.layers import Activation, MaxPool1D, Flatten
from keras.layers.convolutional import Conv1D, Convolution1D, Convolution2D
from keras.utils import np_utils
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adadelta

In [24]:
import os
os.environ["THEANO_FLAGS"] = "OMP_NUM_THREADS=8"

In [58]:
from keras import backend as K

def my_init(shape, dtype=None):
    return K.random_normal(shape, stddev=0.05, dtype=dtype)     

def conv_model(max_words_count, words_set, word2vec_dim, embedding_weights):
    
    word2vec_shape = 400
    inputs = Input(shape=(max_words_count,), dtype="float32", name="inputs_a")
    
    embedding = Embedding(len(words_set) + 1, word2vec_dim, input_length=max_words_count, 
              weights=[embedding_weights], trainable=False, mask_zero=False)(inputs)
    embedding = Dropout(0.5)(embedding)
    l_conv1 = Conv1D(filters=1024, kernel_size=3)(embedding)
    l_pool1 = MaxPool1D(pool_size=3)(l_conv1)
    
    
    #l_conv2 = Conv1D(filters=256, kernel_size=3)(l_pool1)
    #l_pool2 = MaxPool1D(pool_size=3)(l_conv2)
    
    #l_conv3 = Conv1D(filters=256, kernel_size=3)(l_pool2)
    #l_pool3 = MaxPool1D(pool_size=3)(l_conv3)
    
    flat = Flatten()(l_pool1)
    
    fc_layer1 = Dense(3000, activation='relu')(flat)
    fc_layer1 = BatchNormalization()(fc_layer1)
    drop1_out = Dropout(0.5)(fc_layer1)
    
    fc_layer2 = Dense(1000, activation='relu')(drop1_out)
    fc_layer2 = BatchNormalization()(fc_layer2)
    drop1_out = Dropout(0.5)(fc_layer2)
    
    fc_layer3 = Dense(500, activation='relu')(drop1_out)
    fc_layer3 = BatchNormalization()(fc_layer3)
    drop1_out = Dropout(0.5)(fc_layer3)
    
    
    #fc_layer2 = Dense(256, activation='relu')(drop1_out)
        
    outputs = Dense(5, activation='softmax', name='outputs')(drop1_out)
    my_loss = {'outputs': 'categorical_crossentropy'}

    inputs_all = [
                inputs
            ]

    outputs_all = [
                outputs
            ]

    model = Model(inputs=inputs_all, outputs=outputs_all, )
    nadam = Nadam(lr=1e-2)
    #adadelta = Adadelta()
    model.compile(
                optimizer=nadam,
                loss=my_loss,
                loss_weights={'outputs': 1.},
                metrics = ['accuracy']
              )
    model.summary()
    return model

In [60]:
nb_epoch = 35

model = conv_model(max_words_count=max_words_count,
                        words_set=words_set,
                        word2vec_dim=word2vec_dim,
                        embedding_weights=embedding_weights)
model.optimizer.lr.set_value(0.0001)

for epoch_num in range(nb_epoch):
    fit = model.fit(X_train, y_train, batch_size=32, epochs=1, verbose=1, validation_data=(X_test, y_test),)
    res = model.predict(X_test)
    res = res
    print("epoch: ", epoch_num)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs_a (InputLayer)        (None, 30)                0         
_________________________________________________________________
embedding_23 (Embedding)     (None, 30, 400)           15080800  
_________________________________________________________________
dropout_98 (Dropout)         (None, 30, 400)           0         
_________________________________________________________________
conv1d_45 (Conv1D)           (None, 28, 1024)          1229824   
_________________________________________________________________
max_pooling1d_45 (MaxPooling (None, 9, 1024)           0         
_________________________________________________________________
flatten_23 (Flatten)         (None, 9216)              0         
_________________________________________________________________
dense_42 (Dense)             (None, 3000)              27651000  
__________